## Approach

1. Identify available dates and temporal frequency of observations for the given collection using the GHGC API `/stac` endpoint. The collection processed in this notebook is the Wetland Methane Emissions, LPJ-wsl Model data product.
2. Pass the STAC item into the raster API `/stac/tilejson.json `endpoint.
3. Using `folium.plugins.DualMap`, visualize two tiles (side-by-side), allowing time point comparison.
4. After the visualization, perform zonal statistics for a given polygon.

   

## About the Data

Methane (CH₄) emissions from vegetated wetlands are estimated to be the largest natural source of methane in the global CH₄ budget, contributing to roughly one third of the total of natural and anthropogenic emissions. Wetland CH₄ is produced by microbes breaking down organic matter in the oxygen deprived environment of inundated soils. Due to limited data availability, the details of the role of wetland CH₄ emissions have thus far been underrepresented. Using the Earth Observation SIMulator version (LPJ-EOSIM) of the Lund-Potsdam-Jena Dynamic Global Vegetation Model (LPJ-DGVM) global CH₄ emissions from wetlands are estimated at 0.5° x 0.5 degree spatial resolution. By simulating wetland extent and using characteristics of inundated areas, such as wetland soil moisture, temperature, and carbon content, the model provides estimates of CH₄ quantities emitted into the atmosphere. This dataset shows concentrated methane sources from tropical and high latitude ecosystems. The LPJ-EOSIM Wetland Methane Emissions dataset consists of global daily model estimates of terrestrial wetland methane emissions from 1990 to the present, with data added bimonthly. The estimates are regularly used in conjunction with NASA’s Goddard Earth Observing System (GEOS) model to simulate the impact of wetlands and other methane sources on atmospheric methane concentrations, to compare against satellite and airborne data, and to improve understanding and prediction of wetland emissions.

# Installing the Required Libraries
Please run the next cell to install all the required libraries to run the notebook.

In [11]:
%pip install requests
%pip install folium
%pip install rasterstats
%pip install pystac_client

Note: you may need to restart the kernel to use updated packages.
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/Users/vgaur/miniconda3/envs/cmip6/lib/python3.9/site-packages/numpy-1.24.3.dist-info/METADATA'

Note: you may need to restart the kernel to use updated packages.
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/Users/vgaur/miniconda3/envs/cmip6/lib/python3.9/site-packages/numpy-1.24.3.dist-info/METADATA'

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Querying the STAC API

In [12]:
import requests
from folium import Map, TileLayer
from pystac_client import Client

In [13]:
# Provide STAC and RASTER API endpoints
STAC_API_URL = "http://dev.ghg.center/ghgcenter/api/stac"
RASTER_API_URL = "https://dev.ghg.center/ghgcenter/api/raster"

# Please use the collection name similar to the one used in STAC collection.

# Name of the collection for wetland methane monthly emissions. 
collection_name = "lpjeosim-wetlandch4-daygrid-v2"
asset_name = "ensemble-mean-ch4-wetlands-emissions"

In [14]:
# Fetching the collection from STAC collections using appropriate endpoint.
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()
collection

{'id': 'lpjeosim-wetlandch4-daygrid-v2',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://dev.ghg.center/ghgcenter/api/stac/collections/lpjeosim-wetlandch4-daygrid-v2/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://dev.ghg.center/ghgcenter/api/stac/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://dev.ghg.center/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://dev.ghg.center/ghgcenter/api/stac/collections/lpjeosim-wetlandch4-daygrid-v2'}],
 'title': 'Wetland Methane Emissions, LPJ-EOSIM Model v2',
 'assets': None,
 'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
  'temporal': {'interval': [['1990-01-01T00:00:00+00:00',
     '1990-02-21T00:00:00+00:00']]}},
 'license': 'CC0 1.0',
 'keywords': None,
 'providers': [{'url': None,
   'name': 'NASA',
   'roles': None,
   'description': None}],
 'summaries': {'datetime': ['1990-01-0

Examining the contents of our `collection` under `summaries`, we see that the data is available from January 1980 to December 2021. By looking at `dashboard: time density`, we can see that these observations are collected monthly.

In [15]:
def get_item_count(collection_id):
    count = 0
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items"

    while True:
        response = requests.get(items_url)

        if not response.ok:
            print("error getting items")
            exit()

        stac = response.json()
        count += int(stac["context"].get("returned", 0))
        next = [link for link in stac["links"] if link["rel"] == "next"]

        if not next:
            break
        items_url = next[0]["href"]
        temp = items_url.split('/')
        temp.insert(3, 'ghgcenter')
        temp.insert(4, 'api')
        temp.insert(5, 'stac')
        items_url = '/'.join(temp)

    return count

In [16]:
# Check total number of items available
number_of_items = get_item_count(collection_name)
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit=600"
).json()["features"]
print(f"Found {len(items)} items")

Found 52 items


In [17]:
# Examining the first item in the collection
items[0]

{'id': 'lpjeosim-wetlandch4-daygrid-v2-19900221day',
 'bbox': [-180.0, -90.0, 180.0, 90.0],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://dev.ghg.center/ghgcenter/api/stac/collections/lpjeosim-wetlandch4-daygrid-v2'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://dev.ghg.center/ghgcenter/api/stac/collections/lpjeosim-wetlandch4-daygrid-v2'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://dev.ghg.center/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://dev.ghg.center/ghgcenter/api/stac/collections/lpjeosim-wetlandch4-daygrid-v2/items/lpjeosim-wetlandch4-daygrid-v2-19900221day'}],
 'assets': {'era5-ch4-wetlands-emissions': {'href': 's3://ghgc-data-store-dev/lpjwsl-wetlandch4-daygrid-v2-new-units/ERA5/LPJ_EOSIM_L2_CH4e_dch4e_ERA5_19900221day.tif',
   'type': 'image/tiff; application=geotiff; profile=cloud-optimized',
   'roles': ['data', 'l

Below, we enter minimum and maximum values to provide our upper and lower bounds in `rescale_values.`

In [18]:
rescale_values = {'max': 0.0003, 'min': 0.0}

## Exploring Changes in Methane (CH4) Emission Levels Using the Raster API

In this notebook, we will explore the temporal impacts of methane emissions. We will visualize the outputs on a map using `folium`.


In [19]:
# To access the year value from each item more easily, this will let us query more explicity by year and month (e.g., 2020-02)
items = {item["properties"]["datetime"][:10]: item for item in items} 

Now, we will pass the item id, collection name, and `rescaling_factor` to the `Raster API` endpoint. We will do this twice, once for December 2001 and again for December 2021, so we can visualize each event independently.

In [20]:
color_map = "magma" # select the color ramp from matplotlib library.
date1 = '1990-01-01'
date1_tile = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items[date1]['collection']}&item={items[date1]['id']}"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
date1_tile

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://dev.ghg.center/ghgcenter/api/raster/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=lpjeosim-wetlandch4-daygrid-v2&item=lpjeosim-wetlandch4-daygrid-v2-19900101day&assets=ensemble-mean-ch4-wetlands-emissions&color_formula=gamma+r+1.05&colormap_name=magma&rescale=0.0%2C0.0003'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-180.0, -90.0, 180.0, 90.0],
 'center': [0.0, 0.0, 0]}

In [21]:
date2 = '1990-01-30'
date2_tile = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items[date2]['collection']}&item={items[date2]['id']}"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
date2_tile

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://dev.ghg.center/ghgcenter/api/raster/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=lpjeosim-wetlandch4-daygrid-v2&item=lpjeosim-wetlandch4-daygrid-v2-19900130day&assets=ensemble-mean-ch4-wetlands-emissions&color_formula=gamma+r+1.05&colormap_name=magma&rescale=0.0%2C0.0003'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-180.0, -90.0, 180.0, 90.0],
 'center': [0.0, 0.0, 0]}

## Visualizing CH₄ Emissions


In [23]:
# We will import folium to map and folium.plugins to allow side-by-side mapping
import folium
import folium.plugins

# Set initial zoom and center of map for CH₄ Layer
# Centre of map [latitude,longitude]
map_ = folium.plugins.DualMap(location=(34, -118), zoom_start=6)

# Date 1
map_layer_date1 = TileLayer(
    tiles=date1_tile["tiles"][0],
    attr="GHG",
    opacity=0.5,
)
map_layer_date1.add_to(map_.m1)

# Date 2
map_layer_date2 = TileLayer(
    tiles=date2_tile["tiles"][0],
    attr="GHG",
    opacity=0.5,
)
map_layer_date2.add_to(map_.m2)

# visualising the map
map_



# Calculating Zonal Statistics
To perform zonal statistics, first we need to create a polygon. In this use case, we are creating a polygon in Texas (USA).

In [24]:
# Texas, USA
texas_aoi = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                [-95, 29],
                [-95, 33],
                [-104, 33],
                [-104,29],
                [-95, 29]
            ]
        ],
        "type": "Polygon",
    },
}

In [25]:
# We will plug in the coordinates for a location inside the the polygon and a zoom level

import folium

aoi_map = Map(
    tiles="OpenStreetMap",
    location=[
        30,-101
    ],
    zoom_start=6,
)

folium.GeoJson(texas_aoi, name="Texas, USA").add_to(aoi_map)
aoi_map

In [26]:
# Check total number of items available
items = requests.get(
    f"{STAC_API_URL}/collections/{collection_name}/items?limit=600"
).json()["features"]
print(f"Found {len(items)} items")

Found 52 items


In [27]:
# Explore the first item
items[0]

{'id': 'lpjeosim-wetlandch4-daygrid-v2-19900221day',
 'bbox': [-180.0, -90.0, 180.0, 90.0],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://dev.ghg.center/ghgcenter/api/stac/collections/lpjeosim-wetlandch4-daygrid-v2'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://dev.ghg.center/ghgcenter/api/stac/collections/lpjeosim-wetlandch4-daygrid-v2'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://dev.ghg.center/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://dev.ghg.center/ghgcenter/api/stac/collections/lpjeosim-wetlandch4-daygrid-v2/items/lpjeosim-wetlandch4-daygrid-v2-19900221day'}],
 'assets': {'era5-ch4-wetlands-emissions': {'href': 's3://ghgc-data-store-dev/lpjwsl-wetlandch4-daygrid-v2-new-units/ERA5/LPJ_EOSIM_L2_CH4e_dch4e_ERA5_19900221day.tif',
   'type': 'image/tiff; application=geotiff; profile=cloud-optimized',
   'roles': ['data', 'l

In [30]:
# The bounding box should be passed to the geojson param as a geojson Feature or FeatureCollection
def generate_stats(item, geojson):
    result = requests.post(
        f"{RASTER_API_URL}/cog/statistics",
        params={"url": item["assets"][asset_name]["href"]},
        json=geojson,
    ).json()
    return {
        **result["properties"],
        "datetime": item["properties"]["datetime"],
    }

With the function above, we can generate the statistics for the area of interest.

In [33]:
%%time
stats = [generate_stats(item, texas_aoi) for item in items]

In [ ]:
stats[0]

In [ ]:
import pandas as pd


def clean_stats(stats_json) -> pd.DataFrame:
    df = pd.json_normalize(stats_json)
    df.columns = [col.replace("statistics.b1.", "") for col in df.columns]
    df["date"] = pd.to_datetime(df["datetime"])
    return df


df = clean_stats(stats)
df.head(5)

## Visualizing the Data as a Time Series
We can now explore the wetland methane emissions time series (January 1980 – December 2021) available for the Dallas, Texas area of the U.S. We can plot the data set using the code below:

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(20, 10))


plt.plot(
    df["date"],
    df["max"],
    color="red",
    linestyle="-",
    linewidth=0.5,
    label="Max monthly CH₄ emissions",
)

plt.legend()
plt.xlabel("Years")
plt.ylabel("CH4 emissions g/m2")
plt.title("CH4 emission Values for Texas, Dallas (1980-2021)")

In [ ]:
print(items[2]["properties"]["datetime"])

In [ ]:
october_tile = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items[2]['collection']}&item={items[2]['id']}"
    "&assets=ch4-wetlands-emissions"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}",
).json()
october_tile

In [ ]:
# Use bbox initial zoom and map
# Set up a map located w/in event bounds
import folium

aoi_map_bbox = Map(
    tiles="OpenStreetMap",
    location=[
        30,-100
    ],
    zoom_start=8,
)

map_layer = TileLayer(
    tiles=october_tile["tiles"][0],
    attr="GHG", opacity = 0.5
)

map_layer.add_to(aoi_map_bbox)

aoi_map_bbox

## Summary
In this notebook, we have successfully explored, analyzed, and visualized the STAC collection for wetland methane emissions.